In [4]:
from pathlib import Path
import re
import time 
import requests
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from tqdm import tqdm

data_dir_path = Path('data')
if not data_dir_path.exists():
    data_dir_path.mkdir(parents=True)

In [5]:
def download_data(url, data_dir_path):
    
    file_path = data_dir_path.joinpath(Path(url).name)

    data = requests.get(url).content
    with open(file_path, 'wb') as file:
        file.write(data)

url = 'https://raw.githubusercontent.com/KodairaTomonori/ThreeLineSummaryDataset/master/data/train.csv'
download_data(url=url, data_dir_path=data_dir_path)

url = 'https://raw.githubusercontent.com/KodairaTomonori/ThreeLineSummaryDataset/master/data/test.csv'
download_data(url=url, data_dir_path=data_dir_path)

In [10]:
def anti_join(data1, data2, by):

    joined_data = data1.copy()
    target_data = data2.copy()
    target_data['flag_tmp'] = 1

    if type(by) is str:
        by = [by]

    joined_data = pd.merge(
        joined_data, target_data[by + ['flag_tmp']].drop_duplicates(),
        on=by, how='left'
    ).query('flag_tmp.isnull()', engine='python').drop(columns='flag_tmp').copy()

    return joined_data

columns = ['year', 'month', 'category', 'article_id', 'type_label']

articles = pd.DataFrame()
for data_name in ['train.csv', 'test.csv']:
    data = pd.read_csv(data_dir_path.joinpath(data_name))
    tmp = data.columns.tolist()
    if data_name == 'train.csv':
        data.columns = columns[:-1]
        data = pd.concat([
            data, pd.DataFrame([tmp], columns=columns[:-1])
        ], axis=0)
        data['type_label'] = None
    else:
        data.columns = columns
        data = pd.concat([
            data, pd.DataFrame([tmp], columns=columns)
        ], axis=0)
        
    articles = pd.concat([articles, data], axis=0)

articles = articles.assign(
    year=lambda x: x.year.astype(int),
    article_id=lambda x: x.article_id.map(lambda y: re.sub(r'[a-z\.]', '', str(y))).astype(int)
)

if body_data_file_path.exists():
    articles = anti_join(
        articles,
        pd.read_csv(body_data_file_path).assign(article_id=lambda x: x.article_id.astype(int)),
        by='article_id'
    )

NameError: name 'body_data_file_path' is not defined

In [ ]:
waiting_time = 3              # スクレイピングの間隔
n_writing_data = 10000        # 取得する件数 
article_url = 'http://news.livedoor.com/article/detail/{}/'
body_data_file_path = data_dir_path.joinpath('body_data.csv')
summary_data_file_path = data_dir_path.joinpath('summary_data.csv')

target_articles = articles.sort_values(
    'year', ascending=False
).head(min(len(articles), n_writing_data))